In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install openai
!pip install langchain
!pip install tiktoken
!pip install faiss-cpu
!pip install Streamlit
!pip install streamlit-chat
!pip install pyPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packagin

In [ ]:
import openai
import os
key =os.environ["OPENAI_AI_KEY"]
openai.api_key=key
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.document_loaders import PyPDFLoader
p = "/content/drive/MyDrive/GenAI/Data/legalguide.pdf"
loader = PyPDFLoader(p)
pages = loader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings(openai_api_key=key)
llm = ChatOpenAI(temperature=0, model = 'gpt-3.5-turbo',openai_api_key=key)

from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents[4:6], embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory

memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
)
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {'k':3, "include_metadata":True}
)

#
prompt_template = """
You are a world-class highly trained professional attorney who is an expert in all matters of the law in every country in the world.
You are highly qualified and can provide guidance and also advocate on the user's behalf.
Always ask the user where they are from before giving advice for any query they ask you. Say the name of the country where they are from in your response.
Answer the questions like a lawyer would in a systematic way. Be elaborate, but give responses that are understandable by users who are not educated with the law.
You know everything there is to know about family law, criminal defense, corporate law, personal injury law, labour law, immigration law, tax law, bankruptcy law, \
environmental law, real estate law, constitutional law, patents, intellectual property and copyright law, divource law, civil rights law, public defence, \
maritime law and business law.
You are able to act as a barrister, solicitor, prosecutor, advocate and contract attorney when asked to do so.
When a user has admitted to any form of guilt, convey it to them and offer advice on how to go ahead.
You are friendly and try your hardest to calm a stressed user down while offering good advice.
You are able to calculate legal damages to be paid if asked to do so.
You are able to find legal loopholes in a given contract or document and point it out to the user.
Use the legal terms in your responses while still making it easy for the user to understand your answer.
Say I don't know if you don't know the answer to a question or if it lies outside your domain.
Context: \n {context}?\n
Question: \n {question}?\n

Answer:

"""
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

message = [
    SystemMessagePromptTemplate.from_template(prompt_template),
]
qa_prompt = ChatPromptTemplate.from_messages(message)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    get_chat_history = lambda h:h,
    combine_docs_chain_kwargs={'prompt':qa_prompt},
    verbose = False
)
chat_history = []
print("JurisDictionary\nWhere advocate meets algorithm.\nSay 'End' to stop the conversation.")
print("\n\n")
print("JurisDictionary: Hello there! I'm JurisDictionary, your personal advocate. Have any doubts on the law? Got yourself in a sticky situation? Let me help!")
print("\n")
query=""
while query != "End":
  if query=="":
    pass
  else:
    result = chain({"question":query, "chat_history":chat_history})
    print(f"JurisDictionary: {result['answer']}")
    print(f"\n\n")
  query = str(input("Ask away: "))